# Zilliz 云管道

> [Zilliz Cloud Pipelines](https://docs.zilliz.com/docs/pipelines) 将您的非结构化数据转换为可搜索的向量集合，对您的数据进行嵌入、摄取、搜索和删除的链式处理。
> 
> Zilliz Cloud Pipelines 可在 Zilliz Cloud Console 中通过 RESTful API 获取。

本 Notebook 演示了如何准备 Zilliz Cloud Pipelines 并通过 LangChain Retriever 使用它们。

## 准备 Zilliz Cloud 管道

要为 LangChain Retriever 准备管道，您需要在 Zilliz Cloud 中创建并配置服务。

**1. 设置数据库**

- [注册 Zilliz Cloud](https://docs.zilliz.com/docs/register-with-zilliz-cloud)
- [创建集群](https://docs.zilliz.com/docs/create-cluster)

**2. 创建管道**

- [文档的引入、搜索和删除](https://docs.zilliz.com/docs/pipelines-doc-data)
- [文本的引入、搜索和删除](https://docs.zilliz.com/docs/pipelines-text-data)

## 使用 LangChain Retriever

In [1]:
%pip install --upgrade --quiet langchain-milvus

In [1]:
from langchain_milvus import ZillizCloudPipelineRetriever

retriever = ZillizCloudPipelineRetriever(
    pipeline_ids={
        "ingestion": "<YOUR_INGESTION_PIPELINE_ID>",  # skip this line if you do NOT need to add documents
        "search": "<YOUR_SEARCH_PIPELINE_ID>",  # skip this line if you do NOT need to get relevant documents
        "deletion": "<YOUR_DELETION_PIPELINE_ID>",  # skip this line if you do NOT need to delete documents
    },
    token="<YOUR_ZILLIZ_CLOUD_API_KEY>",
)

### 添加文档

您可以 O使用 `add_texts` 或 `add_doc_url` 方法来添加文档，这些方法可以将来自文本列表或预签名/公共 URL 的文档连同相应的元数据一起插入到存储中。

- 如果使用 **文本摄取管道**，则可以使用 `add_texts` 方法，该方法会将一批文本及其相应的元数据插入到 Zilliz Cloud 存储中。

    **参数:**
    - `texts`: 文本字符串列表。
    - `metadata`: 一个键值元数据字典，将作为摄取管道所需的保留字段插入。默认为 None。

In [3]:
# retriever.add_texts(
#     texts = ["example text 1e", "example text 2"],
#     metadata={"<FIELD_NAME>": "<FIELD_VALUE>"}  # skip this line if no preserved field is required by the ingestion pipeline
#     )

- 如果使用 **文档摄取管道**，可以使用 `add_doc_url` 方法，该方法会将来自 url 的文档及相应的元数据插入 Zilliz Cloud 存储。

    **参数:**
    - `doc_url`: 文档 url。
    - `metadata`: 一个键值元数据字典，将作为摄取管道所需的保留字段插入。默认为 None。

以下示例适用于文档摄取管道，该管道需要将 Milvus 版本作为元数据。我们将使用一个[示例文档](https://publicdataset.zillizcloud.com/milvus_doc.md)，该文档描述了如何在 Milvus v2.3.x 中删除实体。

In [5]:
retriever.add_doc_url(
    doc_url="https://publicdataset.zillizcloud.com/milvus_doc.md",
    metadata={"version": "v2.3.x"},
)

{'token_usage': 1247, 'doc_name': 'milvus_doc.md', 'num_chunks': 6}

### 获取相关文档

要查询检索器，您可以使用 `get_relevant_documents` 方法，该方法返回一个 LangChain `Document` 对象列表。

**参数：**
- `query`: 用于查找相关文档的字符串。
- `top_k`: 结果的数量。默认为 10。
- `offset`: 在搜索结果中要跳过的记录数。默认为 0。
- `output_fields`: 在输出中呈现的额外字段。
- `filter`: 用于过滤搜索结果的 Milvus 表达式。默认为 ""。
- `run_manager`: 要使用的回调处理程序。

In [2]:
retriever.get_relevant_documents(
    "Can users delete entities by complex boolean expressions?"
)

[Document(page_content='# Delete Entities\nThis topic describes how to delete entities in Milvus.  \nMilvus supports deleting entities by primary key or complex boolean expressions. Deleting entities by primary key is much faster and lighter than deleting them by complex boolean expressions. This is because Milvus executes queries first when deleting data by complex boolean expressions.  \nDeleted entities can still be retrieved immediately after the deletion if the consistency level is set lower than Strong.\nEntities deleted beyond the pre-specified span of time for Time Travel cannot be retrieved again.\nFrequent deletion operations will impact the system performance.  \nBefore deleting entities by comlpex boolean expressions, make sure the collection has been loaded.\nDeleting entities by complex boolean expressions is not an atomic operation. Therefore, if it fails halfway through, some data may still be deleted.\nDeleting entities by complex boolean expressions is supported only 